In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import necessary libraries
from pathlib import Path
from PIL import Image
import os
from tqdm import tqdm

# Specify the path to the main folder containing images
drive_folder_path = '/content/drive/MyDrive/dtd/images'

# Define a function to read images with a progress bar
def read_images_with_progress(main_folder_path):
    # Create an empty list to store all the images
    all_images = []

    # Iterate through subdirectories, files in the main folder
    for subdir, _, files in os.walk(main_folder_path):
        # Display a progress bar with the subdirectory name
        for filename in tqdm(files, desc=subdir, position=0, leave=True):
            # Check if the file has a valid image extension
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                # Construct the full path to the image file
                file_path = os.path.join(subdir, filename)

                # Open the image using the PIL library
                img = Image.open(file_path)

                # Append the image to the list
                all_images.append(img)

    # Return the list of all read images
    return all_images

# Call the function with the specified main folder path
all_images = read_images_with_progress(drive_folder_path)

# Print the total number of images read
print("Number of images:", len(all_images))

/content/drive/MyDrive/dtd/images: 0it [00:00, ?it/s]
/content/drive/MyDrive/dtd/images/woven: 100%|██████████| 120/120 [00:00<00:00, 905.43it/s]

Number of images: 5640


In [ ]:
# Import necessary libraries
import cv2
import numpy as np
from tqdm import tqdm

# Create an empty list to store color histogram features for all images
all_hist_features = []

# Iterate through all images with a progress bar
for img in tqdm(all_images, desc="Extracting Color Histogram", position=0, leave=True):
    # Create an empty list to store histogram features for the current image
    hist_features = []

    # Convert the PIL image to a NumPy array and change the color space to BGR
    img_array = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    # Iterate through each channel (B, G, R) of the image
    for channel in cv2.split(img_array):
        # Calculate the histogram for the current channel
        # - channel [0]: for the first channel
        # - Mask: None
        # - Number of bins in the histogram: 256 bins representing pixel intensities
        # - Range of pixel: [0, 256]
        hist = cv2.calcHist([channel], [0], None, [256], [0, 256])

        # Flatten the histogram and extend the features list
        hist_features.extend(hist.flatten())

    # Append the histogram features for the current image to the list
    all_hist_features.append(hist_features)

# Convert the list of histogram features to a NumPy array
all_hist_features = np.array(all_hist_features)

# Print the shape of the color histogram features array
print("Color Histogram Features Shape:", all_hist_features.shape)

Extracting Color Histogram: 100%|██████████| 5640/5640 [00:31<00:00, 176.70it/s]


Color Histogram Features Shape: (5640, 768)


In [ ]:
# Import necessary libraries
from skimage import feature
import numpy as np
from tqdm import tqdm

# Create an empty list to store LBP features for all images
all_lbp_features = []

# Set LBP parameters
lbp_radius = 1  # radius around each pixel where neighboring pixels are considered
lbp_num_points = 8 * lbp_radius  # Number of points sampled in the neighborhood, providing granularity to the LBP pattern

# Iterate through all images with a progress bar
for img in tqdm(all_images, desc="Extracting LBP Features", position=0, leave=True):
    # Convert the PIL image to a NumPy array and change the color space to grayscale
    img_array = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

    # Calculate LBP (Local Binary Pattern) features using the 'uniform' method
    lbp = feature.local_binary_pattern(img_array, lbp_num_points, lbp_radius, method='uniform')

    # Compute the histogram of LBP values
    # np.histogram returns the histogram values and bin edges
    lbp_features, _ = np.histogram(lbp.ravel(), bins=np.arange(0, lbp_num_points + 3), range=(0, lbp_num_points + 2))

    # Append the LBP features for the current image to the list
    all_lbp_features.append(lbp_features)

# Convert the list of LBP features to a NumPy array
all_lbp_features = np.array(all_lbp_features)

# Print the shape of the LBP features array
print("LBP Features Shape:", all_lbp_features.shape)

Extracting LBP Features: 100%|██████████| 5640/5640 [04:38<00:00, 20.27it/s]

LBP Features Shape: (5640, 10)


In [ ]:
# Import necessary libraries
from skimage.feature import hog
from skimage import exposure
import numpy as np
from tqdm import tqdm

# Create an empty list to store HOG features for all images
all_hog_features = []

# Set HOG parameters
hog_orientations = 9  # Number of orientation bins in the HOG descriptor
hog_pixels_per_cell = (8, 8)  # Size (in pixels) of a cell in the HOG grid
hog_cells_per_block = (2, 2)  # Number of cells in each block used for normalization and feature extraction

# Iterate through all images with a progress bar
for img in tqdm(all_images, desc="Extracting HOG Features", position=0, leave=True):
    # Convert the PIL image to a NumPy array and change the color space to grayscale
    img_array = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

    # Calculate HOG (Histogram of Oriented Gradients) features
    hog_features, _ = hog(img_array, orientations=hog_orientations, pixels_per_cell=hog_pixels_per_cell,
                          cells_per_block=hog_cells_per_block, visualize=True, block_norm='L2-Hys')

    # Flatten the HOG features and append them to the list
    all_hog_features.append(hog_features.flatten())

# Convert the list of HOG features to a NumPy array
all_hog_features = np.array(all_hog_features)

# Print the shape of the HOG features array
print("HOG Features Shape:", all_hog_features.shape)

Extracting HOG Features: 100%|██████████| 5640/5640 [41:47<00:00,  2.25it/s]

HOG Features Shape: (5640,)



<ipython-input-46-2007a377676d>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_hog_features = np.array(all_hog_features)


In [ ]:
# Import necessary libraries
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
from tqdm import tqdm

# Load the pre-trained VGG16 model with weights pre-trained on ImageNet
vgg16_model = VGG16(weights='imagenet', include_top=False)

# Specify the layer to extract features from
vgg16_layer_name = 'block5_pool'
# Create a new model for feature extraction from the specified layer
vgg16_feature_extractor = Model(inputs=vgg16_model.input, outputs=vgg16_model.get_layer(vgg16_layer_name).output)

# Create an empty list to store VGG16 features for all images
all_vgg16_features = []

# Set input size for VGG16 model
input_size = (224, 224)

# Iterate through all images with a progress bar
for img in tqdm(all_images, desc="Extracting VGG16 Features", position=0, leave=True):
    # Convert the PIL image to a NumPy array
    img_array = image.img_to_array(img)

    # Resize the image to the input size expected by the VGG16 model
    img_array = cv2.resize(img_array, input_size)

    # Add an extra dimension to the image array to match the input shape expected by VGG16
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the input image (e.g., normalize pixel values)
    img_array = preprocess_input(img_array)

    # Use the VGG16 feature extractor to obtain features from the specified layer
    vgg16_features = vgg16_feature_extractor.predict(img_array)

    # Flatten the VGG16 features and append them to the list
    all_vgg16_features.append(vgg16_features.flatten())

# Convert the list of VGG16 features to a NumPy array
all_vgg16_features = np.array(all_vgg16_features)

# Print the shape of the VGG16 features array
print("VGG16 Features Shape:", all_vgg16_features.shape)

Extracting VGG16 Features:   0%|          | 0/5640 [00:00<?, ?it/s]

1/1 [==============================] - 0s 404ms/step


Extracting VGG16 Features:   0%|          | 1/5640 [00:00<44:47,  2.10it/s]

1/1 [==============================] - 0s 188ms/step


Extracting VGG16 Features:   0%|          | 2/5640 [00:00<31:57,  2.94it/s]

...

1/1 [==============================] - 0s 175ms/step


Extracting VGG16 Features: 100%|█████████▉| 5639/5640 [21:29<00:00,  4.36it/s]

1/1 [==============================] - 0s 180ms/step


Extracting VGG16 Features: 100%|██████████| 5640/5640 [21:29<00:00,  4.37it/s]


VGG16 Features Shape: (5640, 25088)


In [ ]:
# Reshape all_vgg16_features to have two dimensions
all_vgg16_features_2d = all_vgg16_features.reshape(-1, 1)

# Combine Features (using Color Histogram and VGG16)
combined_features = np.concatenate((all_hist_features, all_vgg16_features_2d), axis=1)
print("Combined Features Shape:", combined_features.shape)

Combined Features Shape: (5640, 769)


In [ ]:
from sklearn.decomposition import PCA

# Choose the number of components for PCA
num_components = 50  # You can adjust this based on your requirements

# Create PCA instance and fit_transform the data
pca = PCA(n_components=num_components)
pca_result = pca.fit_transform(combined_features)

# Access the explained variance ratio to see how much variance is retained by each component
explained_variance_ratio = pca.explained_variance_ratio_

In [ ]:
# Print the cumulative explained variance
cumulative_explained_variance = explained_variance_ratio.cumsum()
print("Cumulative Explained Variance:", cumulative_explained_variance[-1])

Cumulative Explained Variance: 0.8818223


In [ ]:
# The reduced feature matrix after PCA
reduced_features = pca_result
print("Reduced Feature Matrix Shape:", reduced_features.shape)

Reduced Feature Matrix Shape: (5640, 50)
